In [42]:
import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta

# Initialize Faker
fake = Faker()

# Parameters
num_customers = 500000  # Number of customers
email_styles = ['style_1', 'style_2', 'style_3']
marital_statuses = ['single', 'married', 'divorced']
education_levels = ['primary', 'secondary', 'tertiary']
start_date = datetime(2022, 1, 1)

# Helper functions to generate fake data
def random_age():
    return random.randint(25, 70)  # Age starts from 25

def random_click_purchase():
    click = fake.boolean(chance_of_getting_true=50)  # 50% chance for click
    purchase = fake.boolean(chance_of_getting_true=20) if click else False  # 30% chance for purchase if clicked
    return click, purchase

def random_date(start_date):
    return start_date + timedelta(days=random.randint(0, 365))

# Generate dataset
data = []
for i in range(num_customers):
    customer_no = i + 1
    age = random_age()
    email_style = random.choice(email_styles)
    marital = random.choice(marital_statuses)
    education = random.choice(education_levels)
    click, purchase = random_click_purchase()
    send_date = random_date(start_date)
    click_date = random_date(send_date) if click else None
    purchase_date = random_date(click_date) if purchase else None
    
    data.append([customer_no, age, marital, education, email_style, send_date, click, click_date, purchase, purchase_date])

# Create DataFrame
df = pd.DataFrame(data, columns=['customer_no', 'age', 'marital_status', 'education', 'email_style', 'send_date', 'clicked', 'click_date', 'purchased', 'purchase_date'])

# Preview the dataset
df.head(10)


,customer_no,age,marital_status,education,email_style,send_date,clicked,click_date,purchased,purchase_date
0,1,43,divorced,tertiary,style_1,2022-07-06,False,NaT,False,NaT
1,2,58,single,primary,style_1,2022-08-11,True,2022-10-12,False,NaT
2,3,47,married,primary,style_3,2022-07-05,True,2023-03-24,False,NaT
3,4,46,single,secondary,style_1,2022-08-13,False,NaT,False,NaT
4,5,52,married,secondary,style_2,2022-03-01,True,2022-05-14,False,NaT
5,6,25,single,primary,style_2,2022-02-26,False,NaT,False,NaT
6,7,25,married,primary,style_3,2022-06-08,True,2022-08-23,True,2023-05-22
7,8,67,single,tertiary,style_2,2022-01-24,True,2022-09-26,True,2023-05-07
8,9,25,single,tertiary,style_3,2022-02-01,True,2022-04-16,False,NaT
9,10,25,single,primary,style_3,2022-11-22,True,2023-08-11,False,NaT


In [43]:
df.education.value_counts()

education
tertiary     166825
primary      166682
secondary    166493
Name: count, dtype: int64

In [44]:
df.marital_status.value_counts()

marital_status
divorced    167130
single      166535
married     166335
Name: count, dtype: int64

In [45]:
df.purchased.value_counts()

purchased
False    449481
True      50519
Name: count, dtype: int64

In [46]:
# Helper functions to calculate CTR and purchase rates by age group, marital status, education, and email style
def calculate_performance(df):
    performance = df.groupby(['age', 'marital_status', 'education', 'email_style']).agg(
        total_sent=('customer_no', 'count'),
        total_clicks=('clicked', 'sum'),
        total_purchases=('purchased', 'sum')
    ).reset_index()

    performance['CTR'] = performance['total_clicks'] / performance['total_sent']
    performance['PurchaseRate'] = performance['total_purchases'] / performance['total_sent']
    return performance

# Calculate initial performance
initial_performance = calculate_performance(df)
initial_performance.head(10)
#print("Initial performance metrics:\n", initial_performance)


,age,marital_status,education,email_style,total_sent,total_clicks,total_purchases,CTR,PurchaseRate
0,25,divorced,primary,style_1,386,202,46,0.523316,0.119171
1,25,divorced,primary,style_2,386,203,35,0.525907,0.090674
2,25,divorced,primary,style_3,425,219,37,0.515294,0.087059
3,25,divorced,secondary,style_1,386,199,33,0.515544,0.085492
4,25,divorced,secondary,style_2,390,221,47,0.566667,0.120513
5,25,divorced,secondary,style_3,381,189,26,0.496063,0.068241
6,25,divorced,tertiary,style_1,439,208,46,0.473804,0.104784
7,25,divorced,tertiary,style_2,403,211,50,0.523573,0.124069
8,25,divorced,tertiary,style_3,379,192,31,0.506596,0.081794
9,25,married,primary,style_1,413,202,40,0.489104,0.096852


In [47]:
# Function to get best-performing email style based on CTR and Purchase Rate
def get_best_email_styles(df):
    performance = calculate_performance(df)

    # Get the email style with the highest CTR for each group of age, marital status, and education
    best_by_ctr = performance.loc[performance.groupby(['age', 'marital_status', 'education'])['CTR'].idxmax()]
    best_by_ctr = best_by_ctr[['age', 'marital_status', 'education', 'email_style', 'CTR']]
    best_by_ctr = best_by_ctr.rename(columns={'email_style': 'best_email_style_ctr', 'CTR': 'best_ctr'})
    
    # Get the email style with the highest Purchase Rate for each group of age, marital status, and education
    best_by_purchase_rate = performance.loc[performance.groupby(['age', 'marital_status', 'education'])['PurchaseRate'].idxmax()]
    best_by_purchase_rate = best_by_purchase_rate[['age', 'marital_status', 'education', 'email_style', 'PurchaseRate']]
    best_by_purchase_rate = best_by_purchase_rate.rename(columns={'email_style': 'best_email_style_purchase_rate', 'PurchaseRate': 'best_purchase_rate'})

    # Merge both results into one DataFrame
    best_styles = pd.merge(best_by_ctr, best_by_purchase_rate, on=['age', 'marital_status', 'education'])

    return best_styles

# Calculate and display the best email styles
best_email_styles = get_best_email_styles(df)
best_email_styles.head(10)


,age,marital_status,education,best_email_style_ctr,best_ctr,best_email_style_purchase_rate,best_purchase_rate
0,25,divorced,primary,style_2,0.525907,style_1,0.119171
1,25,divorced,secondary,style_2,0.566667,style_2,0.120513
2,25,divorced,tertiary,style_2,0.523573,style_2,0.124069
3,25,married,primary,style_2,0.551724,style_1,0.096852
4,25,married,secondary,style_3,0.495050,style_1,0.140845
5,25,married,tertiary,style_2,0.537349,style_2,0.122892
6,25,single,primary,style_1,0.526699,style_2,0.097744
7,25,single,secondary,style_1,0.539379,style_2,0.108466
8,25,single,tertiary,style_2,0.520455,style_3,0.107050
9,26,divorced,primary,style_3,0.536199,style_2,0.126582


In [49]:
import pandas as pd
import numpy as np
from faker import Faker
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Initialize Faker for data simulation
fake = Faker()
np.random.seed(42)

# 1. Generate synthetic dataset with multiple marketing strategies
def generate_synthetic_data(n=5000):
    data = []
    for _ in range(n):
        age = np.random.randint(18, 70)  # Age range from 18 to 70
        marital_status = np.random.choice(['single', 'married', 'divorced', 'widowed'])
        education = np.random.choice(['primary', 'secondary', 'tertiary'])
        
        # Marketing strategy variables
        email_style = np.random.choice(['style_1', 'style_2', 'style_3'])
        sms_frequency = np.random.choice(['weekly', 'bi-weekly', 'monthly'])
        phone_call_frequency = np.random.choice(['none', 'quarterly', 'monthly'])
        social_media_platform = np.random.choice(['Facebook', 'Instagram', 'LinkedIn', 'Twitter'])
        
        # Campaign details for each marketing strategy
        total_sent_email = np.random.randint(100, 500)  # Emails sent between 100 and 500
        total_sent_sms = np.random.randint(50, 300)  # SMS sent between 50 and 300
        total_calls = np.random.randint(0, 50) if phone_call_frequency != 'none' else 0  # Calls range based on frequency
        total_social_media_ads = np.random.randint(100, 1000)  # Social media ads between 100 and 1000
        
        # Simulated interaction data for each strategy
        total_clicks_email = np.random.randint(0, total_sent_email)
        total_clicks_sms = np.random.randint(0, total_sent_sms)
        total_clicks_social_media = np.random.randint(0, total_social_media_ads)
        
        # Purchases are influenced by multiple channels
        total_purchases = np.random.randint(0, total_clicks_email + total_clicks_sms + total_clicks_social_media + 1)
        
        # Financial metrics
        customer_lifetime_value = np.random.uniform(100, 10000)  # CLV between $100 and $10,000
        campaign_cost_email = np.random.uniform(0.1, 5.0) * total_sent_email  # Email campaign cost
        campaign_cost_sms = np.random.uniform(0.05, 3.0) * total_sent_sms  # SMS campaign cost
        campaign_cost_calls = np.random.uniform(1.0, 10.0) * total_calls  # Phone call cost
        campaign_cost_social_media = np.random.uniform(0.1, 2.0) * total_social_media_ads  # Social media ad cost
        
        total_campaign_cost = (campaign_cost_email + campaign_cost_sms + campaign_cost_calls + campaign_cost_social_media)
        
        data.append({
            'age': age,
            'marital_status': marital_status,
            'education': education,
            'email_style': email_style,
            'sms_frequency': sms_frequency,
            'phone_call_frequency': phone_call_frequency,
            'social_media_platform': social_media_platform,
            'total_sent_email': total_sent_email,
            'total_sent_sms': total_sent_sms,
            'total_calls': total_calls,
            'total_social_media_ads': total_social_media_ads,
            'total_clicks_email': total_clicks_email,
            'total_clicks_sms': total_clicks_sms,
            'total_clicks_social_media': total_clicks_social_media,
            'total_purchases': total_purchases,
            'customer_lifetime_value': customer_lifetime_value,
            'total_campaign_cost': total_campaign_cost
        })
    
    return pd.DataFrame(data)

# Generate the dataset
df = generate_synthetic_data()

# 2. Calculate CTR, Purchase Rate, and ROI
def calculate_performance_and_roi(df):
    df['CTR_email'] = df['total_clicks_email'] / df['total_sent_email']
    df['CTR_sms'] = df['total_clicks_sms'] / df['total_sent_sms']
    df['CTR_social_media'] = df['total_clicks_social_media'] / df['total_social_media_ads']
    df['PurchaseRate'] = df['total_purchases'] / (df['total_sent_email'] + df['total_sent_sms'] + df['total_social_media_ads'])
    
    # Revenue from total purchases based on CLV
    df['total_revenue'] = df['total_purchases'] * df['customer_lifetime_value']
    # ROI calculation
    df['ROI'] = (df['total_revenue'] - df['total_campaign_cost']) / df['total_campaign_cost']
    
    return df[['age', 'marital_status', 'education', 'email_style', 'sms_frequency', 'phone_call_frequency', 
               'social_media_platform', 'total_sent_email', 'total_sent_sms', 'total_calls', 'total_social_media_ads', 
               'total_clicks_email', 'total_clicks_sms', 'total_clicks_social_media', 'CTR_email', 'CTR_sms', 
               'CTR_social_media', 'PurchaseRate', 'customer_lifetime_value', 'total_campaign_cost', 
               'total_revenue', 'ROI']]

# Calculate performance and ROI
performance_df = calculate_performance_and_roi(df)
performance_df.head()



,age,marital_status,education,email_style,sms_frequency,phone_call_frequency,social_media_platform,total_sent_email,total_sent_sms,total_calls,...,total_clicks_sms,total_clicks_social_media,CTR_email,CTR_sms,CTR_social_media,PurchaseRate,customer_lifetime_value,total_campaign_cost,total_revenue,ROI
0,56,widowed,primary,style_3,monthly,none,Facebook,202,171,0,...,87,372,0.366337,0.508772,0.657244,0.105431,1514.381497,1767.375815,1.499238e+05,83.828460
1,19,widowed,primary,style_1,bi-weekly,quarterly,Facebook,335,138,48,...,14,189,0.173134,0.101449,0.329268,0.166189,6222.021492,2313.377230,1.082632e+06,466.987549
2,24,single,primary,style_3,bi-weekly,none,Twitter,113,291,0,...,52,339,0.787611,0.178694,0.386986,0.071094,4457.509688,2119.580066,4.056334e+05,190.374409
3,53,married,secondary,style_2,bi-weekly,quarterly,Instagram,359,103,28,...,43,161,0.604457,0.417476,0.200748,0.159019,8958.790769,1802.066135,1.800717e+06,998.251309
4,57,single,primary,style_2,monthly,none,Twitter,316,237,0,...,156,14,0.126582,0.658228,0.029228,0.042636,838.051372,2164.386342,3.687426e+04,16.036820


In [50]:
# 3. Build and train a predictive model for ROI optimization
# Encode categorical variables for modeling
df_encoded = pd.get_dummies(performance_df, columns=['marital_status', 'education', 'email_style', 'sms_frequency', 
                                                     'phone_call_frequency', 'social_media_platform'])

# Prepare features and target variable
X = df_encoded.drop(columns=['ROI', 'total_revenue', 'total_sent_email', 'total_sent_sms', 'total_calls', 
                             'total_social_media_ads', 'total_clicks_email', 'total_clicks_sms', 
                             'total_clicks_social_media', 'CTR_email', 'CTR_sms', 'CTR_social_media', 'PurchaseRate'])
y = df_encoded['ROI']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
r2_score = model.score(X_test, y_test)
print("Model R^2 Score:", r2_score)

# Show predictions for a few test samples
y_pred = model.predict(X_test)
predicted_roi = pd.DataFrame({'Predicted ROI': y_pred[:5], 'Actual ROI': y_test[:5].values})
print(predicted_roi)


Model R^2 Score: 0.25973802338606666
   Predicted ROI   Actual ROI
0    1745.532173  1476.051072
1     136.874344   108.858822
2     983.562238   786.364078
3     143.924511    73.472162
4     455.461009   588.166696


In [51]:
# Dynamic adjustment function: Increase email sends for best-performing email style by age, marital status, and education
def adjust_campaign(df, performance):
    adjustments = {}
    
    # For each combination of age, marital status, and education, find the best-performing email style
    for _, group in performance.groupby(['age', 'marital_status', 'education']):
        best_email_style = group.loc[group['CTR'].idxmax(), 'email_style']
        
        # Create a key for adjustments based on age, marital status, and education
        key = (group['age'].iloc[0], group['marital_status'].iloc[0], group['education'].iloc[0])
        adjustments[key] = best_email_style
    
    # Adjust email styles based on the new strategy
    def adjust_email_style(row):
        key = (row['age'], row['marital_status'], row['education'])
        if key in adjustments:
            return adjustments[key]
        return row['email_style']
    
    df['adjusted_email_style'] = df.apply(adjust_email_style, axis=1)
    return df

# Apply adjustments and recalculate performance
adjusted_df = adjust_campaign(df.copy(), initial_performance)
adjusted_performance = calculate_performance(adjusted_df)

# Compare pre- and post-adjustment metrics
#print("\nAdjusted performance metrics:\n", adjusted_performance)


KeyError: "Column(s) ['clicked', 'customer_no', 'purchased'] do not exist"